Manipulación de datos con el ORM
====

* Última actualización: 12 de septiembre de 2022.

Preparación
-----

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()


class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship(
        "Address", back_populates="user", cascade="all, delete-orphan"
    )

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [2]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=False,
    future=True,
)

In [3]:
Base.metadata.create_all(engine)

In [4]:
from sqlalchemy.orm import Session

with Session(engine) as session:

    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )

    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )

    patrick = User(
        name="patrick",
        fullname="Patrick Star",
    )

    session.add_all(
        [
            spongebob,
            sandy,
            patrick,
        ]
    )

    session.commit()

Adición de objetos a una sesión
----

In [6]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

In [7]:
squidward

User(id=None, name='squidward', fullname='Squidward Tentacles')

In [8]:
krabs

User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')

In [9]:
session = Session(engine)
session.add(squidward)
session.add(krabs)

#
# Los objetos pendientes por insertar se ven en session.new
#
session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

In [10]:
#
# Cuando se emiten los cambios a la BD se hace un flush.
#
session.flush()

In [11]:
#
# No hay cambios pendientes
#
session.new

IdentitySet([])

Claves primarias autogeneradas
----

In [12]:
squidward.id

4

In [13]:
krabs.id

5

Obtención de objetos por la clave primaria
---

In [14]:
some_squidward = session.get(User, 4)
some_squidward

User(id=4, name='squidward', fullname='Squidward Tentacles')

In [15]:
some_squidward is squidward

True

Commiting
---

In [ ]:
session.commit()

Actualización de objetos
----

In [16]:
from sqlalchemy import select

sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()
sandy

User(id=2, name='sandy', fullname='Sandy Cheeks')

In [17]:
#
# Se altera el atributo de un objeto
#
sandy.fullname = "Sandy Squirrel"

In [18]:
#
# El objeto pertenece a una colección "dirty"
#
sandy in session.dirty

True

In [19]:
sandy_fullname = session.execute(select(User.fullname).where(User.id == 2)).scalar_one()

print(sandy_fullname)

Sandy Squirrel


In [20]:
sandy in session.dirty

False

UPDATE en el ORM
---

In [21]:
from sqlalchemy import update

session.execute(
    update(User)
    .where(User.name == "sandy")
    .values(fullname="Sandy Squirrel Extraordinaire")
)

In [22]:
sandy.fullname

'Sandy Squirrel Extraordinaire'

Borrado de objetos ORM
----

In [23]:
patrick = session.get(User, 3)

In [24]:
session.delete(patrick)

In [25]:
session.execute(select(User).where(User.name == "patrick")).first()

In [26]:
patrick in session

False

In [27]:
from sqlalchemy import delete

squidward = session.get(User, 4)

session.execute(delete(User).where(User.name == "squidward"))

In [28]:
squidward in session

False

Rolling back
-----

In [29]:
session.rollback()

In [30]:
sandy.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1123a7a30>}

In [31]:
sandy.fullname

'Sandy Cheeks'

In [32]:
sandy.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1123a7a30>,
 'id': 2,
 'fullname': 'Sandy Cheeks',
 'name': 'sandy'}

In [33]:
patrick in session

True

In [34]:
session.execute(select(User).where(User.name == "patrick")).scalar_one() is patrick

True

Cerrado de una sesión
---

In [35]:
session.close()